# Clustering Experiment

We have put all of our methods together for direct comparison in a single notebook.

Problem: Currently we are determining only the indiviudal digits, but we need to recognize these as coherent numbers and be able to assign entries to numbers.


### Register Images to Start

To start, we need to register images using the `utilities/conversion/apply_homography_to_labels.ipynb` notebook. This should be run before running this notebook. This notebook is built on the assumption that the `data/registered_images` directory has been created and populated. Additionally it assumes that the `data/yolo_data.json` file is created. Both of these are created in the referenced notebook.


#### Install Packages

These are the necessary packages to run the functions and scripts below.


In [43]:
# Standard libraries
import os
import json
import random
from pathlib import Path
from typing import List, Tuple, Dict

# Third-party libraries
import cv2
import numpy as np
from PIL import Image, ImageDraw
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde

# Local libraries
from utils.annotations import BoundingBox

#### Start By Loading YOLO Data


To start I want to bring in the YOLO formatted data for each sheet and I can additionally load the respective images. As mentioned above you must have ran the `utilities/conversion/apply_homography_to_labels.ipynb` notebook to generate this YOLO data.


In [44]:
# Load yolo_data.json
PATH_TO_YOLO_DATA = "../../data/yolo_data.json"
PATH_TO_REGISTERED_IMAGES = "../../data/registered_images"
UNIFIED_IMAGE_PATH = (
    "../../data/unified_intraoperative_preoperative_flowsheet_v1_1_front.png"
)

# Load yolo_data.json
with open(PATH_TO_YOLO_DATA) as json_file:
    yolo_data = json.load(json_file)

# See how many intraoperative images are registered
print(f"Found {len(yolo_data)} sheets in yolo_data.json")

# Load the json for bp and hr cluster locations
PATH_TO_CLUSTER_LOCATIONS = "../../data/bp_and_hr_cluster_locations.json"
with open(PATH_TO_CLUSTER_LOCATIONS) as json_file:
    bp_hr_cluster_locations = json.load(json_file)
    print(f"Found {len(bp_hr_cluster_locations)} items in bp_and_hr_cluster_locations.json")

Found 19 sheets in yolo_data.json
Found 19 items in bp_and_hr_cluster_locations.json


#### Define Constants Used In Notebook

In [45]:
DESIRED_IMAGE_WIDTH = 800
DESIRED_IMAGE_HEIGHT = 600

Now let's select relevant bounding boxes from the blood pressure and HR zone.

Start by defining functions to convert YOLO bounding box format to pixels (to see if the bounding box is within region of interest). Then create a function that allows you to select ROI and returns a list of bounding boxes within this ROI.


In [46]:
def find_density_max(values: List[int], search_area: int) -> int:
    """
    Given a list of values and a search area, find the index of where the highest density is.
    The list of values correspond to identifying points for the bounding boxes and the search area corresponds to the images height or width.

    Args:
        values: List of identifying points for the bounding boxes
        search_area: height/width of the image dependent on whether x or y axis is being search.

    Returns:
        The axis value that has the highest density of bounding boxes.
    """
    kde = gaussian_kde(values, bw_method=0.2)

    x_values = np.linspace(0, search_area, 10000)

    kde_vals = kde(x_values)

    max_index = np.argmax(kde_vals)
    return x_values[max_index]

def remove_bb_outliers(boxes: List[BoundingBox]) -> List[BoundingBox]:
    """
    Given a list of bounding boxes, remove the outliers from the x axis, then remove the outliers from the y axis

    Args:
        boxes: List of Bounding Boxes to filter

    Returns:
        Filtered list of Bounding Boxes
    """
    x_vals = [bb.left for bb in boxes]
    # find the 25th percentile
    x_Q1 = np.percentile(x_vals, 25)
    # find the 75th percentile
    x_Q3 = np.percentile(x_vals, 75)
    # find the IQR
    x_IQR = x_Q3 - x_Q1
    # determine lower and upper bounds
    x_lower = x_Q1 - 1.5*x_IQR
    x_upper = x_Q3 + 1.5*x_IQR
    # remove outliers via the x axis
    x_filtered = [bb for bb in boxes if x_lower <= bb.left <= x_upper]

    y_vals = [bb.top for bb in x_filtered]
    # find the 25th percentile
    y_Q1 = np.percentile(y_vals, 25)
    # find the 75th percentile
    y_Q3 = np.percentile(y_vals, 75)
    # find the IQR
    y_IQR = y_Q3 - y_Q1
    # determine the lower and upper bounds
    y_lower = y_Q1 - 1.5*y_IQR
    y_upper = y_Q3 + 1.5*y_IQR
    # remove outliers via the y axis
    filtered = [bb for bb in x_filtered if y_lower <= bb.top <= y_upper]

    return x_filtered



def select_relevant_bounding_boxes(
    sheet_data: List[str],
    path_to_image: Path,
    show_images: bool = False,
    desired_img_width: int = DESIRED_IMAGE_WIDTH,
    desired_img_height: int = DESIRED_IMAGE_HEIGHT,
) -> Tuple[List[str], List[str]]:
    """
    Given sheet data for bounding boxes in YOLO format, find the bounding boxes corresponding to the number and time on the BP chart.
    Return the bounding boxes that are within the selected region split into two lists: time labels and numerical values.

    Args:
        sheet_data: List of bounding boxes in YOLO format.
        path_to_image: Path to the image file.

    Returns:
        Tuple of Lists of string representations of bounding boxes that are within the selected region, in YOLO format.
        The first list contains bounding boxes in the top-right region -- representing time labels.
        The second list contains bounding boxes in the bottom-left region -- representing numerical values for mmHg and bpm.
            (bounding_boxes_time, bounding_boxes_numbers)
    """

    # Load the image
    image = cv2.imread(path_to_image)

    # Display the image and allow the user to select a ROI
    resized_image = cv2.resize(image, (desired_img_width, desired_img_height))

    # convert the YOLO data to Bounding Boxes
    bboxes: List[BoundingBox] = [
        BoundingBox.from_yolo(yolo_bb, desired_img_width, desired_img_height)
        for yolo_bb in sheet_data
    ]

    # generate a list of the digit categories
    digit_categories: List[str] = [str(i) for i in range(10)]

    # filter out non bounding boxes and those whose category is not a digit
    bboxes: List[BoundingBox] = list(
        filter(
            lambda bb: isinstance(bb, BoundingBox) and bb.category in digit_categories,
            bboxes,
        )
    )

    # find the point with the maximum density of bounding boxes
    bboxes_right: List[int] = [bb.right for bb in bboxes]
    # x_loc is the vertical line to the left of the time axis and right of the numbers axis
    x_loc: int = find_density_max(bboxes_right, desired_img_width)

    bboxes_bottom: List[int] = [bb.bottom for bb in bboxes]
    # y_loc is the horizontal line undert the time axis and above the number axis
    y_loc: int = find_density_max(bboxes_bottom, desired_img_height)


    bounding_boxes_time = []
    bounding_boxes_numbers = []

    # Process the bounding boxes
    for bounding_box in bboxes:
        # get the center point of the bounding box for comparison
        x_center_bb, y_center_bb = bounding_box.center

        # check if the bounding box is a number on the BP chart by comparing to the KDE index + a threshold
        if (x_center_bb > x_loc-15 and x_center_bb < x_loc+2):
            bounding_boxes_numbers.append(bounding_box)
        # check if the bounding box is a time on the BP chart by comparing to the KDE index + a threshold
        elif (y_center_bb > y_loc-10 and y_center_bb < y_loc+2):
            bounding_boxes_time.append(bounding_box)
    
    bounding_boxes_numbers = remove_bb_outliers(bounding_boxes_numbers)
    bounding_boxes_time = remove_bb_outliers(bounding_boxes_time)

    for bounding_box in bounding_boxes_numbers:
        x_min = int(bounding_box.left)
        x_max = int(bounding_box.right)
        y_min = int(bounding_box.top)
        y_max = int(bounding_box.bottom)

        # Bounding box is in the top-right region
        cv2.rectangle(
            resized_image, (x_min, y_min), (x_max, y_max), (255, 255, 0), 1
        )
    
    for bounding_box in bounding_boxes_time:
        x_min = int(bounding_box.left)
        x_max = int(bounding_box.right)
        y_min = int(bounding_box.top)
        y_max = int(bounding_box.bottom)

        cv2.rectangle(
            resized_image, (x_min, y_min), (x_max, y_max), (255, 0, 255), 1
        )
            
    # plot the lines of the KDE index found for debugging
    # numbers_start = (int(x_loc), 0)
    # numbers_end = (int(x_loc), desired_img_height)

    # time_start = (0, int(y_loc))
    # time_end = (desired_img_width, int(y_loc))

    # cv2.line(resized_image, numbers_start, numbers_end, (255,255,0), 1)
    # cv2.line(resized_image, time_start, time_end, (255,0,255), 1)

    # Close all OpenCV windows, always do this or it will annoyingly not go away
    # You can also manually quit out with ESC key.
    cv2.destroyAllWindows()

    # If we are showing the images, display the image with the selected region and bounding boxes
    # Bounding boxes in the top-right region (time) are in one color while those in the bottom left (numerical) are in another
    if show_images:
        # Display the image with the selected region and bounding boxes
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        resized_image = Image.fromarray(resized_image)
        resized_image.show()

    # Return a tuple of bounding boxes in the top-right and bottom-left regions
    return (bounding_boxes_time, bounding_boxes_numbers)

Create functions for K-means clustering, dbscan clustering, and agglomerative clustering


In [47]:
def cluster_kmeans(
    bounding_boxes: List[BoundingBox], possible_nclusters: List[int]
) -> List[int]:
    """
    Cluster bounding boxes using K-Means clustering algorithm.

    Args:
        bounding_boxes: List of bounding boxes in YOLO format.
        possible_nclusters: List of possible number of clusters to try.

    Returns:
        List of cluster labels.
    """
    # Convert to a NumPy array (using only x_center and y_center)
    data = np.array([box.center for box in bounding_boxes])

    cluster_performance_map = {}
    for number_of_clusters in possible_nclusters:
        if number_of_clusters > len(data):
            raise (
                f"Number of clusters {number_of_clusters} is greater than number of bounding boxes {len(data)}."
            )
        if number_of_clusters < 1:
            raise (f"Number of clusters {number_of_clusters} must be greater than 0.")
        # Apply K-Means
        kmeans = KMeans(
            n_clusters=number_of_clusters,
            init="k-means++",
            n_init=20,
            max_iter=500,
            tol=1e-8,
            random_state=42,
        )
        kmeans.fit(data)

        # Get cluster labels
        labels = kmeans.predict(data)
        silhouette_avg = silhouette_score(data, labels)

        # print(
        #     f"Number of clusters: {number_of_clusters}, Silhouette score: {silhouette_avg}"
        # )

        cluster_performance_map[number_of_clusters] = {
            "score": silhouette_avg,
            "labels": labels,
        }

    # Evaluate the performance of each number of clusters and select the one with the highest silhouette score
    # if it is 0.003 greater than what should be the number of clusters otherwise go with proper_nclusters
    n_clusters_max_silhouette = max(
        cluster_performance_map, key=lambda x: cluster_performance_map[x]["score"]
    )
    best_n_clusters = (
        n_clusters_max_silhouette
        if (
            (
                cluster_performance_map[n_clusters_max_silhouette]["score"]
                - cluster_performance_map[max(possible_nclusters)]["score"]
            )
            >= 0.005
        )
        else max(possible_nclusters)
    )
    return cluster_performance_map[best_n_clusters]["labels"]


def dbscan_clustering(
    bounding_boxes: List[BoundingBox], defined_eps: float, min_samples: int
) -> List[int]:
    """
    Cluster bounding boxes density based spatial clustering algorithm.

    Args:
        bounding_boxes: List of bounding boxes.
        defined_eps: Maximum distance between two samples to be in the neighborhood of one another (center of BB).
        min_samples: The number of samples (or total weight) for a point to be considered as core

    Returns:
        List of cluster labels.
    """
    # Convert to a NumPy array (using only x_center and y_center)
    data = np.array([box.center for box in bounding_boxes])

    # DBSCAN
    scan = DBSCAN(eps=defined_eps, min_samples=min_samples)
    labels = scan.fit_predict(data)

    return labels


def agglomerative_clustering(
    bounding_boxes: List[BoundingBox], possible_nclusters: List[int]
) -> List[int]:
    # make the bonding box data into a Numpy array
    data = np.array([box.center for box in bounding_boxes])

    # follow suit of the cluster_kmeans algorithm to measure accuracy through silhoutte scores
    cluster_performance_map = {}
    for number_of_clusters in possible_nclusters:
        if number_of_clusters > len(data):
            raise (
                f"Number of clusters {number_of_clusters} is greater than number of bounding boxes {len(data)}."
            )
        if number_of_clusters < 1:
            raise (f"Number of clusters {number_of_clusters} must be greater than 0.")
        # use agglomerative clustering
        agg = AgglomerativeClustering(n_clusters=number_of_clusters, linkage="single")
        # get labels
        labels = agg.fit_predict(data)
        # compute the silhoutte scores
        silhouette_avg = silhouette_score(data, labels)

        cluster_performance_map[number_of_clusters] = {
            "score": silhouette_avg,
            "labels": labels,
        }

    # get the number of clusters with the best silhoutte score
    n_clusters_max_silhouette = max(
        cluster_performance_map, key=lambda x: cluster_performance_map[x]["score"]
    )

    best_n_clusters = (
        n_clusters_max_silhouette
        if (
            (
                cluster_performance_map[n_clusters_max_silhouette]["score"]
                - cluster_performance_map[max(possible_nclusters)]["score"]
            )
            >= 0.003
        )
        else max(possible_nclusters)
    )
    return cluster_performance_map[best_n_clusters]["labels"]


Code block to get the average X and Y of  the expected clusters. We get the average of the location of each label across sheets.

In [48]:
cluster_locations_dict = {} # Dictionary containing the cluster_name as the key to another dictionary with 'x' and 'y' as keys for lists of x and y coordinates
for sheet_num, data in enumerate(yolo_data.items()):
    # For each sheet, get the cluster center X and Y coordinates and add them to the cluster_locations_dict dictionary
    expected_clusters = bp_hr_cluster_locations[sheet_num]
    for cluster in expected_clusters["annotations"][0]["result"]:
            x_expected_perc, y_expected_perc = cluster['value']['x'], cluster['value']['y'] # Get the expected cluster location (percent x and y in the original image space)
            x_expected, y_expected = (x_expected_perc/100) * DESIRED_IMAGE_WIDTH, (y_expected_perc/100) * DESIRED_IMAGE_HEIGHT # Convert the expected cluster location to pixel space
            cluster_name = cluster['value']['rectanglelabels'][0]
            if cluster_name not in cluster_locations_dict:
                cluster_locations_dict[cluster_name] = {'x': [], 'y': []}
            cluster_locations_dict[cluster_name]['x'].append(x_expected)
            cluster_locations_dict[cluster_name]['y'].append(y_expected)

# Average the cluster locations
for cluster_name, cluster_data in cluster_locations_dict.items():
    cluster_locations_dict[cluster_name]['x'] = float(np.mean(cluster_data['x']))
    cluster_locations_dict[cluster_name]['y'] = float(np.mean(cluster_data['y']))

Function to create a result dictionary that we can save as a JSON file to analyze performance.


In [49]:
def create_result_dictionary(
    labels: List[str], bounding_boxes: List[BoundingBox]
) -> Dict[int, int]:
    """
    Create a dictionary with cluster labels as keys and lists of bounding boxes as values.

    Args:
        labels: List of cluster labels.
        bounding_boxes: List of bounding boxes.
        expected_clusters: dictionary containing expected clustered
            - This should be passed as specific for the sheet we are working with. It can be obtained from bp_hr_cluster_locations.json.

    Returns:
        Dictionary with cluster labels as keys and bounding box values as values.
    """
    # Create a dictionary to store labelled elements
    label_dict = {}

    # Iterate over both lists for labels and the bounding boxes found
    for label, box in zip(labels, bounding_boxes):
        label = int(label)
        if label not in label_dict:
            # Create a new list for this label if it doesn't exist
            label_dict[label] = []
        # Append the element to the corresponding label list
        label_dict[label].append(box)

    # So now we have a dictionary with the clusters as keys and a list of bounding box objects as strings as values

    # Now we want to impute meaning of these clusters based on how close they are to their expected cluster locations
    for key in label_dict:
        # For each key get the centers of the bounding boxes, and compute the middle point
        x_centers, y_centers  = [element.center[0] for element in label_dict[key]], [element.center[1] for element in label_dict[key]]
        x_found, y_found = sum(x_centers) / len(x_centers), sum(y_centers) / len(y_centers)
        # Now we have the center point of the cluster
        # We will use the euclidean distance to determine the closest expected cluster location and use that as the label
        distances = [] # List that contains the distances for each expected cluster from our found cluster
        for cluster in cluster_locations_dict:
            x_expected, y_expected = cluster_locations_dict[cluster]['x'], cluster_locations_dict[cluster]['y']
            #print(f"Cluster location: {x}, {y}")
            distance = np.sqrt((x_expected - x_found) ** 2 + (y_expected - y_found) ** 2)
            #print(f"Distance: {distance}")
            distances.append(distance)
        # Get the index of the minimum distance
        min_distance_index = distances.index(min(distances))

        # Get the label of the cluster
        label_dict[key] = list(cluster_locations_dict.keys())[min_distance_index]

        # Add the distance to the dictionary
        label_dict[key] = {"label": label_dict[key], "distance": min(distances)}

    return label_dict

Function to generate colors!


In [50]:
# Draw bounding boxes on the image
def generate_color():
    return "#%06x" % random.randint(0, 0xFFFFFF)

Function to generate random Bounding Box formatted occurances.


In [51]:
def random_time_generate(x):
    # erroneous bounding boxes for time ROI
    category_int = random.randint(0, 9)
    left_int = random.randint(127, 715)

    # slope and random points constrained to time (x) axis
    top_int = random.uniform(222, 234)

    # input generated integers to bounding box
    box = BoundingBox(
        category=f"{category_int}",
        left=left_int,
        top=top_int,
        right=left_int + 4,
        bottom=top_int + 6,
    )
    return box


def random_number_generate(x):
    # erroneous bounding boxes for number ROI
    category_int = random.randint(0, 9)
    left_int = random.randint(108, 117)

    # slope and random points constrained to number (y) axis
    top_int = random.uniform(235, 411)

    # input generated integers to bounding box
    box = BoundingBox(
        category=f"{category_int}",
        left=left_int,
        top=top_int,
        right=left_int + 4,
        bottom=top_int + 6,
    )
    return box

Function to remove 5% bounding boxes and create 5% erroneous.


In [52]:
def erroneous_bounding_boxes(
    time_BB: List[str], number_BB: List[str]
) -> Tuple[List[str], List[str]]:
    """
    Create 5% erroneous bounding boxes by simultaneously removing and generating time and number BB.

    Args:
        time_BB: list of time bounding boxes in BoundingBox format
        number_BB: list of number bounding boxes in BoundingBox format

    Returns:
        Tuple: lists of time and number bounding boxes in BoundingBox format
    """
    # make copies of input bounding box lists to avoid unwanted manipulation
    time_BB_copy = time_BB.copy()
    number_BB_copy = number_BB.copy()

    # subset and remove 5% of bounding boxes from time/number_bounding_boxes lists
    ## sample
    time_BB_sample = list(random.sample(time_BB_copy, 4))
    number_BB_sample = list(random.sample(number_BB_copy, 4))
    ## remove
    _ = [time_BB_copy.remove(line) for line in time_BB_sample]
    _ = [number_BB_copy.remove(line) for line in number_BB_sample]

    # use random bounding box generation to refill removed BBs with erroneous boxes
    time_BB_generate = list(map(random_time_generate, range(len(time_BB_sample))))
    number_BB_generate = list(map(random_number_generate, range(len(number_BB_sample))))

    # append BB generated list back to copy with 5% removal
    time_BB_erroneous = time_BB_copy + time_BB_generate
    number_BB_erroneous = number_BB_copy + number_BB_generate

    return (time_BB_erroneous, number_BB_erroneous)

### Function that tests the clustering methods with our without erroneous boxes

Now lets use these functions to get the relevant bounding boxes for clustering.


In [53]:
def test_clustering_methods(add_erroneous = True) -> None:
    """
    Test the clustering methods on the YOLO data.
    Saves the clustered images and the clustered bounding boxes to JSON files.

    Args:
        add_erroneous: Boolean flag to add erroneous bounding boxes to the data.

    Returns:
        None
    """
    sheet_num = 0
    # Iterate over all images and their bounding boxes
    for sheet, yolo_bbs in yolo_data.items():
        #print(f"Sheet: {sheet}")
        full_image_path = os.path.join(PATH_TO_REGISTERED_IMAGES, sheet)
        #print(f"Full image path: {full_image_path}")

        # Call the analyze_sheet function with data from the loop
        time_bounding_boxes, number_bounding_boxes = select_relevant_bounding_boxes(
            yolo_bbs, full_image_path
        )

        if add_erroneous:
            # make erroneous bounding boxes -- simultaneously add and remove %5 of boxes
            time_bounding_boxes, number_bounding_boxes = erroneous_bounding_boxes(
                time_bounding_boxes, number_bounding_boxes
            )

        for method in ["kmeans", "dbscan", "agglomerative"]:
            # Now we need to cluster the bounding boxes that pertain to the same multi-digit number
            if method == "kmeans":
                time_labels = cluster_kmeans(time_bounding_boxes, [40, 41, 42])
                number_labels = cluster_kmeans(number_bounding_boxes, [18, 19, 20])
            elif method == "dbscan":
                time_labels = dbscan_clustering(
                    time_bounding_boxes, defined_eps=5, min_samples=1
                )
                number_labels = dbscan_clustering(
                    number_bounding_boxes, defined_eps=5, min_samples=2
                )
            elif method == "agglomerative":
                time_labels = agglomerative_clustering(
                    time_bounding_boxes, [40, 41, 42]
                )
                number_labels = agglomerative_clustering(
                    number_bounding_boxes, [18, 19, 20]
                )
            else:
                raise ValueError(f"Invalid clustering method: {method}")

            # Create an image object
            image: Image = Image.open(full_image_path)
            image_width, image_height = image.size

            label_color_map = {}
            for i, label in enumerate(time_labels):
                # Get the bounding box
                bounding_box = time_bounding_boxes[i]
                x_min, y_min, x_max, y_max = [
                    (coor / 800) * image_width
                    if i % 2 == 0
                    else (coor / 600) * image_height
                    for i, coor in enumerate(bounding_box.box)
                ]

                # If the label is not in the color map, generate a new color
                if label not in label_color_map:
                    label_color_map[label] = generate_color()

                # Open the image
                draw = ImageDraw.Draw(image)

                draw.rectangle(
                    [
                        x_min,
                        y_min,
                        x_max,
                        y_max,
                    ],
                    outline=label_color_map[label],
                    width=3,
                )

            # Save the image with the bounding boxes to the kmeans_clustered_images folder
            image.save(f"../../data/{method}_clustered_images/time/{sheet}")

            # Save the clustered bounding boxes to a JSON file
            with open(
                f"../../data/{method}_clustered_images/results/time/{sheet.split('.')[0]}.json",
                "w",
            ) as f:
                json.dump(
                    create_result_dictionary(time_labels, time_bounding_boxes),
                    f,
                )

            # Create an image object
            image: Image = Image.open(full_image_path)
            image_width, image_height = image.size
            label_color_map = {}
            for i, label in enumerate(number_labels):
                # Get the bounding box
                bounding_box = number_bounding_boxes[i]
                x_min, y_min, x_max, y_max = [
                    (coor / 800) * image_width
                    if i % 2 == 0
                    else (coor / 600) * image_height
                    for i, coor in enumerate(bounding_box.box)
                ]

                # If the label is not in the color map, generate a new color
                if label not in label_color_map:
                    label_color_map[label] = generate_color()

                # Open the image
                draw = ImageDraw.Draw(image)

                draw.rectangle(
                    [
                        x_min,
                        y_min,
                        x_max,
                        y_max,
                    ],
                    outline=label_color_map[label],
                    width=3,
                )

            # Save the image with the bounding boxes to the kmeans_clustered_images folder
            image.save(f"../../data/{method}_clustered_images/number/{sheet}")

            # Save the clustered bounding boxes to a JSON file
            with open(
                f"../../data/{method}_clustered_images/results/number/{sheet.split('.')[0]}.json",
                "w",
            ) as f:
                json.dump(
                    create_result_dictionary(
                        number_labels, number_bounding_boxes
                    ),
                    f,
                )

        sheet_num += 1

#### Analyze accuracy

Below we write a function that analyzes the accuracy of our clustering methods.


In [54]:
def analyze_accuracy():
    # Since this work is done above, we can simply read in from the JSON files created in the previous step and work from there.
    for method in ["kmeans", "dbscan", "agglomerative"]:
        print(f"Method: {method}")
        # Paths to the JSON files
        PATH_TO_RESULTS = f"../../data/{method}_clustered_images/results"
        TIME_JSON = os.path.join(PATH_TO_RESULTS, "time")
        NUMBER_JSON = os.path.join(PATH_TO_RESULTS, "number")

        time_wrong_clusters_count = 0
        time_correct_clusters_count = 0
        number_wrong_clusters_count = 0
        number_correct_clusters_count = 0

        # Iterate over all images and their bounding boxes
        number_distance_values = []
        time_distance_values = []
        number_distance_values_erroneous = []
        time_distance_values_erroneous = []
        # Undetected clusters
        undetected_time_clusters = []
        undetected_number_clusters = []
        for sheet, yolo_bb in yolo_data.items():
            full_image_path = os.path.join(PATH_TO_REGISTERED_IMAGES, sheet)
            time_bounding_boxes, number_bounding_boxes = select_relevant_bounding_boxes(
                yolo_bb, full_image_path,
            )
            # Convert the bounding boxes to a list of strings with proper suffixes
            expected_time_values = [
                "0_mins",
                "5_mins",
                "10_mins",
                "15_mins",
                "20_mins",
                "25_mins",
                "30_mins",
                "35_mins",
                "40_mins",
                "45_mins",
                "50_mins",
                "55_mins",
                "60_mins",
                "65_mins",
                "70_mins",
                "75_mins",
                "80_mins",
                "85_mins",
                "90_mins",
                "95_mins",
                "100_mins",
                "105_mins",
                "110_mins",
                "115_mins",
                "120_mins",
                "125_mins",
                "130_mins",
                "135_mins",
                "140_mins",
                "145_mins",
                "150_mins",
                "155_mins",
                "160_mins",
                "165_mins",
                "170_mins",
                "175_mins",
                "180_mins",
                "185_mins",
                "190_mins",
                "195_mins",
                "200_mins",
                "205_mins",
            ]

            expected_number_values = [
                "30_mmhg",
                "40_mmhg",
                "50_mmhg",
                "60_mmhg",
                "70_mmhg",
                "80_mmhg",
                "90_mmhg",
                "100_mmhg",
                "110_mmhg",
                "120_mmhg",
                "130_mmhg",
                "140_mmhg",
                "150_mmhg",
                "160_mmhg",
                "170_mmhg",
                "180_mmhg",
                "190_mmhg",
                "200_mmhg",
                "210_mmhg",
                "220_mmhg",
            ]

            # Load JSON
            with open(os.path.join(TIME_JSON, f"{sheet.split('.')[0]}.json")) as f:
                time_clusters = json.load(f)

            # Each cluster contains the number (integer) that the cluster represents
            # We know what integers should be represented in the time labels, lets check that they are all there.
            # Keep track of any false positives (new clusters that don't exist) or negatives (missing clusters)
            for cluster, value in time_clusters.items():
                if value['label'] not in expected_time_values:
                    # Print the sheet, value that is not in the expected values
                    # print(f"Time -> Sheet: {sheet}, Value: {value["label"]}.")
                    # We have an erroneous cluster
                    time_wrong_clusters_count += 1
                    time_distance_values_erroneous.append(value['distance'])
                else:
                    # We have a correct cluster
                    expected_time_values.remove(value['label'])
                    time_correct_clusters_count += 1
                    time_distance_values.append(value['distance'])
            
            undetected_time_clusters += expected_time_values

            # Load JSON
            with open(os.path.join(NUMBER_JSON, f"{sheet.split('.')[0]}.json")) as f:
                number_clusters = json.load(f)

            # Each cluster contains the number (integer) that the cluster represents
            # We know what integers should be represented in the time labels, lets check that they are all there.
            # Keep track of any false positives (new clusters that don't exist) or negatives (missing clusters)
            for cluster, value in number_clusters.items():
                if value['label'] not in expected_number_values:
                    #print(f"Number -> Sheet: {sheet}, Value: {value}")
                    # We have an erroneous cluster
                    number_wrong_clusters_count += 1
                    number_distance_values_erroneous.append(value['distance'])
                else:
                    # We have a correct cluster
                    expected_number_values.remove(value['label'])
                    number_correct_clusters_count += 1
                    number_distance_values.append(value['distance'])

            undetected_number_clusters += expected_number_values

        print(
            f"Time labels: {time_correct_clusters_count} correct clusters, {time_wrong_clusters_count} incorrect clusters. There were {len(undetected_time_clusters)} undetected clusters. The accuracy is {(time_correct_clusters_count - (time_wrong_clusters_count + len(undetected_time_clusters))) / (42 * 19) * 100:.2f}%.\nAverage distance when correct: {sum(time_distance_values) / len(time_distance_values):.2f}px, incorrect: {sum(time_distance_values_erroneous) / 1 if len(time_distance_values_erroneous) == 0 else len(time_distance_values_erroneous):.2f}px, and overall: {sum(time_distance_values + time_distance_values_erroneous) / (len(time_distance_values) + len(time_distance_values_erroneous)):.2f}px"
        )
        print("\n")
        print(
            f"Number labels: {number_correct_clusters_count} correct clusters, {number_wrong_clusters_count} incorrect clusters. There were {len(undetected_number_clusters)} undetected clusters. The accuracy is {(number_correct_clusters_count - (number_wrong_clusters_count + len(undetected_number_clusters))) / (20 * 19) * 100:.2f}%.\nAverage distance when correct: {sum(number_distance_values) / len(number_distance_values):.2f}px, incorrect: {sum(number_distance_values_erroneous) / 1 if len(number_distance_values_erroneous) == 0 else len(number_distance_values_erroneous):.2f}px, and overall: {sum(number_distance_values + number_distance_values_erroneous) / (len(number_distance_values) + len(number_distance_values_erroneous)):.2f}px"
        )
        print("\n\n")


### Test without Erroneous bounding boxes

In [ ]:
# Test the clustering methods with errouneous bounding boxes
test_clustering_methods(add_erroneous=False)
analyze_accuracy()

### Test with Erroneous bounding boxes

In [ ]:
# Test the clustering methods with errouneous bounding boxes
test_clustering_methods(add_erroneous=True)
analyze_accuracy()

5.128868424650202
3.597980694608979
3.6395575952532355
4.804956720638203
5.085709814009798
5.329439138658003
5.468683100585791
5.3653974835934015
5.359311360003873
5.51992258407943
4.03895502689332
5.384104017654249
5.47355122243425
3.7985818772779707
3.7220368158846755
5.091528957051915
5.0637997966833606
5.3084852757707885
5.172937368260436
5.074207897853961
5.172965966723361
5.310827116685281
5.149645919955512
3.346670391093665
3.765323134093238
3.2225625389035732
3.296491342258328
3.655187371229006
6.163149911474913
7.0523907056064
5.0474097355117395
4.987380699121186
5.160089775698283
2.756527305864094
5.155577319890164
3.272783971815679
5.0020983710973566
3.3291181611952267
3.3173945481904363
4.100111132930009
4.733050052242177
5.19139906849865
6.787741066894071
6.796769937103355
6.808282011027717
6.405934970964202
6.418908222762696
4.7380342199726835
6.770903409927329
6.547973925461767
6.640841808964941
7.0419992144740515
6.568469141542965
6.34003775721372
4.829954576148932
4.90

KeyboardInterrupt: 